## Import libraries

In [3]:
import numpy as np
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [4]:
#import tensorflow as tf
import numpy as np
#from tensorflow.keras import layers, models
import keras
from keras.models import Sequential 
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix

In [5]:
import librosa

## Import data

In [6]:
outcome = np.load("outcome.npy")
specs = np.load("specs.npy")

In [7]:
normalised_specs =  librosa.util.normalize(specs, norm=2, axis=1, threshold=1.0)

## Preprocess

In [8]:
X = normalised_specs
y = outcome

#X = np.vstack(X)
X = np.expand_dims(X, -1)
y = np.expand_dims(y, -1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [9]:
X.shape

(860, 128, 79, 1)

In [10]:
def outcome_reshape(input):
    out = []
    for i in range(len(input)):
        if input[i][0] == True: #normal is true
            out.append(0) #0
        else:
            out.append(1)
    return out

In [11]:
def model_evaluation(model, X_train, X_test, y_train, y_test, threshold = 0.5):
    score = model.evaluate(X_train, y_train, verbose=0)
    print("Train set:")
    print('Train loss:', score[0])
    print('Train accuracy:', score[1])
    
    pred_proba_train = model.predict(X_train)
    pred_train = tf.greater(pred_proba_train, threshold)
    pred_train_reshape = outcome_reshape(pred_train)
    y_train_reshape = outcome_reshape(y_train)
    tn, fp, fn, tp = confusion_matrix(y_train_reshape, pred_train_reshape, labels = [0,1]).ravel()
    print("tn:", tn, "fp:", fp, "fn:", fn, "tp:", tp )
    print("Sensitivity:", tp/(tp+fn), "| Specificity:", tn/(tn+fp))
    print("\n")

    print("Test set:")
    score = model.evaluate(X_test, y_test, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])

    pred_proba_test = model.predict(X_test)
    pred_test = tf.greater(pred_proba_test, threshold)
    pred_test_reshape = outcome_reshape(pred_test)
    y_test_reshape = outcome_reshape(y_test)
    tn, fp, fn, tp = confusion_matrix(y_test_reshape, pred_test_reshape, labels = [0,1]).ravel()
    print("tn:", tn, "fp:", fp, "fn:", fn, "tp:", tp )
    print("Sensitivity:", tp/(tp+fn), "| Specificity:", tn/(tn+fp))



## Model - CNN

In [43]:
##model building
input_shape = (X_train.shape[1], X_train.shape[2], 1)
num_category = 2
model = Sequential()
#convolutional layer with rectified linear unit activation
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
#32 convolution filters used each of size 3x3
#again
model.add(Conv2D(64, (3, 3), activation='relu'))
#64 convolution filters used each of size 3x3
#choose the best features via pooling
model.add(MaxPooling2D(pool_size=(2, 2)))
#randomly turn neurons on and off to improve convergence
model.add(Dropout(0.25))
#flatten since too many dimensions, we only want a classification output
model.add(Flatten())
#fully connected to get all relevant data
model.add(Dense(128, activation='relu'))
#one more dropout for convergence' sake :) 
model.add(Dropout(0.5))
#output a softmax to squash the matrix into output probabilities
model.add(Dense(num_category, activation='sigmoid'))

#Adaptive learning rate (adaDelta) is a popular form of gradient descent rivaled only by adam and adagrad
#categorical ce since we have multiple classes (10) 
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

c:\Users\rishi\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [44]:
batch_size = 64
num_epoch = 4
#model training
model_log = model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=num_epoch,
          verbose=1,
          validation_data=(X_test, y_test))

Epoch 1/4


11/11 ━━━━━━━━━━━━━━━━━━━━ 13s 966ms/step - accuracy: 0.5000 - loss: 0.6937 - val_accuracy: 0.5000 - val_loss: 0.6933
Epoch 2/4
11/11 ━━━━━━━━━━━━━━━━━━━━ 11s 997ms/step - accuracy: 0.5000 - loss: 0.6934 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 3/4
11/11 ━━━━━━━━━━━━━━━━━━━━ 11s 974ms/step - accuracy: 0.5000 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6930
Epoch 4/4
11/11 ━━━━━━━━━━━━━━━━━━━━ 10s 925ms/step - accuracy: 0.5000 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6928


In [52]:
model_evaluation(model, X_train, X_test, y_train, y_test, threshold = 0.50)

Train set:
Train loss: 0.6928572654724121
Train accuracy: 0.5
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 65ms/step
tn: 471 fp: 106 fn: 94 tp: 17
Sensitivity: 0.15315315315315314 | Specificity: 0.8162911611785095


Test set:
Test loss: 0.6927960515022278
Test accuracy: 0.5
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
tn: 122 fp: 27 fn: 19 tp: 4
Sensitivity: 0.17391304347826086 | Specificity: 0.8187919463087249


In [47]:
print("abnormal: ", sum(y_train_reshape), "normal: ", len(y_train_reshape) - sum(y_train_reshape))

abnormal:  111 normal:  577


## Lenet

In [56]:
##model building
input_shape = (X_train.shape[1], X_train.shape[2], 1)
num_category = 2
model = Sequential()
#convolutional layer with rectified linear unit activation
model.add(Conv2D(20, kernel_size=(5, 5),
                 activation='relu',
                 strides= (1, 1),
                 input_shape=input_shape))
#32 convolution filters used each of size 3x3
#again

#64 convolution filters used each of size 3x3
#choose the best features via pooling
model.add(MaxPooling2D(pool_size=(2, 2)))
#randomly turn neurons on and off to improve convergence
model.add(Conv2D(50, (5, 5), strides= (1, 1), activation='relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
#fully connected to get all relevant data
model.add(Dense(128, activation='relu'))

#output a softmax to squash the matrix into output probabilities
model.add(Dense(num_category, activation='sigmoid'))

#Adaptive learning rate (adaDelta) is a popular form of gradient descent rivaled only by adam and adagrad
#categorical ce since we have multiple classes (10) 
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy', 
            #   tf.keras.metrics.FalsePositives(),
            #   tf.keras.metrics.FalseNegatives(),
            #   tf.keras.metrics.TruePositives(),
            #   tf.keras.metrics.TrueNegatives(),
              'BinaryAccuracy',
              'recall'])

c:\Users\rishi\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [57]:
batch_size = 64
num_epoch = 3
#model training
model_log = model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=num_epoch,
          verbose=1,
          validation_data=(X_test, y_test))

Epoch 1/3


11/11 ━━━━━━━━━━━━━━━━━━━━ 4s 254ms/step - BinaryAccuracy: 0.5103 - accuracy: 0.5000 - loss: 0.6932 - recall: 0.0880 - val_BinaryAccuracy: 0.6512 - val_accuracy: 0.5000 - val_loss: 0.6931 - val_recall: 0.3547
Epoch 2/3
11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 222ms/step - BinaryAccuracy: 0.6361 - accuracy: 0.5000 - loss: 0.6930 - recall: 0.3727 - val_BinaryAccuracy: 0.7093 - val_accuracy: 0.5000 - val_loss: 0.6929 - val_recall: 0.5174
Epoch 3/3
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 204ms/step - BinaryAccuracy: 0.7208 - accuracy: 0.5000 - loss: 0.6929 - recall: 0.5519 - val_BinaryAccuracy: 0.7529 - val_accuracy: 0.5000 - val_loss: 0.6927 - val_recall: 0.6163


In [63]:
model_evaluation(model, X_train, X_test, y_train, y_test, threshold = 0.5)

Train set:
Train loss: 0.692749559879303
Train accuracy: 0.7434592843055725
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step
tn: 423 fp: 154 fn: 85 tp: 26
Sensitivity: 0.23423423423423423 | Specificity: 0.733102253032929


Test set:
Test loss: 0.6927259564399719
Test accuracy: 0.7529069781303406
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
tn: 106 fp: 43 fn: 17 tp: 6
Sensitivity: 0.2608695652173913 | Specificity: 0.7114093959731543


In [24]:
print("abnormal: ", sum(y_train_reshape), "normal: ", len(y_train_reshape) - sum(y_train_reshape))

abnormal:  111 normal:  577


In [46]:
## new model